In [1]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [2]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [3]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

c:\Users\Gamer\miniconda3\Lib\site-packages\charset_normalizer\api.py:105: UserWarning: Trying to detect encoding from a tiny portion of (23) byte(s).
  warn('Trying to detect encoding from a tiny portion of ({}) byte(s).'.format(length))
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MATAS.CO?modules=financialData&corsDomain=finance.yahoo.com&formatted=false&symbol=MATAS.CO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MATAS.CO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MATAS.CO&crumb=Edge%3A+Too+Many+Requests


Error with MATAS.CO: Expecting value: line 1 column 1 (char 0). Removed Ticker from list.


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TRIFOR.CO?modules=financialData&corsDomain=finance.yahoo.com&formatted=false&symbol=TRIFOR.CO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TRIFOR.CO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TRIFOR.CO&crumb=Edge%3A+Too+Many+Requests


Error with TRIFOR.CO: Expecting value: line 1 column 1 (char 0). Removed Ticker from list.


KeyboardInterrupt: 